# Edinburgh bike counter

Arjan Geers

Analysis of [Edinburgh bike counter data](https://data.edinburghopendata.info/dataset/bike-counter-data-set-cluster).

## Preamble

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

from bikecounter.data import get_edinburgh_bike_counter_data

In [ ]:
%matplotlib inline
plt.style.use('seaborn')

## Get data

In [ ]:
data = get_edinburgh_bike_counter_data()

In [ ]:
data.tail()

## Bike counter activity

Total number of active bike counters per month.

In [ ]:
monthly = data.resample('M').sum()
monthly.where(monthly > 0.0).count(axis=1).plot();

Number of bikes per month for each bike counter. It appears that:
1. There are two periods in which few counters were active
2. A different set of counters was active from early 2015 onward than before

In [ ]:
monthly.where(monthly > 0.0).plot(colormap='viridis', legend=False);

Date ranges of bike counter activity.

In [ ]:
for street_counter, street in enumerate(monthly.columns):
    active_months = monthly[street].where(monthly[street] > 0).dropna().index
    for i in range(0, len(active_months)-1):
        if active_months[i+1].to_period('M') - active_months[i].to_period('M') == 1:
            # Draw hline for each consecutive pair of active months
            plt.hlines(street_counter,
                       active_months[i],
                       active_months[i+1])
plt.yticks(range(0, len(monthly.columns), 3), monthly.columns[::3]);